## Rating base recommendation system

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import os
from scipy.sparse import coo_matrix

In [19]:
#read dataset
df = pd.read_csv(r'cleaned_train_data.csv')
df.columns

Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'rating', 'rating_count', 'user_id', 'product_link',
       'about_product'],
      dtype='object')

In [21]:
average_ratings = df.groupby(['product_id','product_name','actual_price','category','about_product','product_link','rating_count'])['rating'].mean().reset_index()

In [23]:
top_rated_items = average_ratings.sort_values(by='rating', ascending=False)

rating_base_recommendation = top_rated_items.head(10)

In [25]:
rating_base_recommendation.loc[:,'rating'] = rating_base_recommendation['rating'].astype(int)
rating_base_recommendation.loc[:,'rating_count'] = rating_base_recommendation['rating_count'].astype(int)

In [27]:
print("Rating Base Recommendation System: (Trending Products)")
rating_base_recommendation[['product_id','product_name','actual_price','category','about_product','product_link','rating_count']] = rating_base_recommendation[['product_id','product_name','actual_price','category','about_product','product_link','rating_count']]
rating_base_recommendation

Rating Base Recommendation System: (Trending Products)


C:\Users\Mindulee Fonseka\AppData\Local\Temp\ipykernel_38496\1993256587.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_base_recommendation[['product_id','product_name','actual_price','category','about_product','product_link','rating_count']] = rating_base_recommendation[['product_id','product_name','actual_price','category','about_product','product_link','rating_count']]


,product_id,product_name,actual_price,category,about_product,product_link,rating_count,rating
1342,1342,Syncwire LTG to USB Cable for Fast Charging Co...,1999.0,Computers&Accessories|Accessories&Peripherals|...,This sturdy and durable cable made of tpe and ...,https://www.amazon.in/Syncwire-Cable-Charging-...,5.0,5.0
1349,1349,"REDTECH USB-C to Lightning Cable 3.3FT, [Apple...",999.0,Computers&Accessories|Accessories&Peripherals|...,💎[The Fastest Charge] - This iPhone USB C cabl...,https://www.amazon.in/REDTECH-Lightning-Certif...,0.0,5.0
1121,1121,Amazon Basics Wireless Mouse | 2.4 GHz Connect...,1000.0,Computers&Accessories|Accessories&Peripherals|...,Reliable wireless connection up to 10m|Advance...,https://www.amazon.in/Wireless-Connection-Batt...,23.0,5.0
1350,1350,Swiffer Instant Electric Water Heater Faucet T...,1999.0,"Home&Kitchen|Heating,Cooling&AirQuality|WaterH...",✔Quick Electric Hot Water Tap Heating tube: hi...,https://www.amazon.in/Swiffer-Instant-Electric...,53803.0,4.0
1205,1205,"Instant Pot Air Fryer, Vortex 2QT, Touch Contr...",20049.0,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,VOLTAGE (230 Volts): Instant Vortex Air Fryer ...,https://www.amazon.in/Instant-Vortex-2QT-EvenC...,3964.0,4.0
1348,1348,"Oratech Coffee Frother electric, milk frother ...",499.0,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,-Make delicious milk foam creamer for your dri...,https://www.amazon.in/Oratech-electric-cappucc...,28.0,4.0
1343,1343,Campfire Spring Chef Prolix Instant Portable W...,3500.0,"Home&Kitchen|Heating,Cooling&AirQuality|WaterH...",Prolix Auto cut off feature: It is available w...,https://www.amazon.in/Spring-Chef-Stainless-Re...,2591.0,4.0
1145,1145,Spigen EZ Fit Tempered Glass Screen Protector ...,2899.0,Electronics|Mobiles&Accessories|MobileAccessor...,Compatibility ; Glass Screen Protector Compati...,https://www.amazon.in/Spigen-Tempered-Screen-P...,7779.0,4.0
1078,1078,Sony Bravia 164 cm (65 inches) 4K Ultra HD Sma...,139900.0,"Electronics|HomeTheater,TV&Video|Televisions|S...",Resolution: 4K Ultra HD (3840 x 2160) | Refres...,https://www.amazon.in/Sony-Bravia-inches-Googl...,5935.0,4.0
1318,1318,Zuvexa USB Rechargeable Electric Foam Maker - ...,1299.0,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,The food taste stainless steel whisking head i...,https://www.amazon.in/Zuvexa-Rechargeable-Elec...,54.0,4.0


In [29]:
# Specify the file path where you want to save the cleaned data
output_file_path = 'top_rated_products.csv'

# Save the cleaned DataFrame to a CSV file
rating_base_recommendation.to_csv(output_file_path, index=False)